###**Sudoku Game Solver: One Agent, Many Constraints**

Simple program to solve any **sudoku** problem (https://en.wikipedia.org/wiki/Sudoku). This program plays the game as a computer player using AI algorithms. It accomplishes the following:
  - Starts the game with a random 4 x 4 sudoku board input by the user (we said 9 x 9 in Exercise 7 but it is just too time consuming and meaningless to make such a big board. As an AI, if it has intelligence to solve 4 x 4 sudoku, then it should be able to also solve 9 x 9 sudoku since the constraints are the same we just need to change a few numbers)
  - Tries to use all 7 uninformed and informed search algorithms in the lecture individually to solve the problem (some just don't work, couldn't be implemented)
  - Stops when the problem is solved,and the unique solution reached, prints out the final solution

### **Define the game logic and all relevant functions**
  1. `validMove(board, move)` checks if the move is valid or not, is also the function that introduces most of the rules to the agent
  2. `createBoard()` creates a blank board waiting for the user to input the initial state in
  3. `printBoard(board)` prints out the current state of the board intput to this function
  4. `updateBoard(board, position, symbol)` takes in the board, the position on the board and the number to be put into that position
  5. `inputBoard(board)` gets user input for the initial state of the problem
  


In [ ]:
def create_board():
    ''' Creates a blank 4x4 Sudoku board
    '''
    board = [[" " for _ in range(4)] for _ in range(4)]
    return board


def print_board(board):
    ''' Prints the 4x4 Sudoku board
    '''
    print()
    for i in range(4):
        print(" ", end="")
        for j in range(4):
            print(board[i][j], end=" ")
            if (j + 1) % 2 == 0 and j != 3:
                print("|", end=" ")
        print()
        if (i + 1) % 2 == 0 and i != 3:
            print("----+----")
    print()

def update_board(board, row, col, symbol):
    ''' Updates the 4x4 Sudoku board
    '''
    if board[row][col] == " ":
        board[row][col] = symbol
    else:
        print("Position already filled. Please choose another position.")

#=====================================================================================#

#functions for the humans to input the initial state of the problem

def inputBoard(board):
    choice = 0
    while choice != -1:
        row = int(input("Input the row (1-4): ")) - 1  # Adjusting for zero-based indexing
        column = int(input("Input the column (1-4): ")) - 1  # Adjusting for zero-based indexing
        number = int(input("Input the number (1-4): "))
        if 0 <= row <= 3 and 0 <= column <= 3 and 1 <= number <= 4:
            update_board(board, row, column, str(number))
            choice = int(input("To continue input, choose 0. To stop input, choose -1: "))
        else:
            print("Invalid input. Please input row, column, and number in the range 1-4.")

    print("Input stopped.")


#=====================================================================================#

#so this sets the rules
def valid_move(board, move):
  valid = True
  row, col, num = move
  num = str(num)
  for i in range(4):
    if (board[row][i] == num):
      valid = False
      return valid
  for j in range(4):
    if (board[j][col] == num):
      valid = False
      return valid

  # Check if the number already exists in the 2x2 square
  start_row = (row // 2) * 2  # Find the starting row index of the square
  start_col = (col // 2) * 2  # Find the starting column index of the square
  for i in range(start_row, start_row + 2):
      for j in range(start_col, start_col + 2):
          if board[i][j] == num:
            valid = False
            return valid

  return valid


### **The search algorithms:**

### 1. Breadth first search

- Cannot be implemented
- It is not suited for CSP problems such as this naturally
- Doesn't take into account the fact that sudoku requires rows, columns and blocks can't have the same number
- Suppose the agent starts at a filled slot and does BFS first on all the slots around it. As it does the same thing to the slots it searches by nature, the slots will all be filled with the same number which violates the rules of sudoku

### 2. Uniform-cost search
 - Uniform-cost search expands nodes based on their path cost from the start node. However, in Sudoku, the path cost between nodes is uniform since each step or transition between states (placing a number in a cell) has the same cost. Hence, using uniform-cost search in Sudoku would essentially reduce to a breadth-first or depth-first search because all transitions have the same cost.

### 3. Depth-first search

 - First defines a stack data structure with method push(adds to the top of the stack), pop(removes the top item from the stack), and is_empty(tells if the stack is really empty or not)
 - `is_board_solved` helper function checks if the sudoku problem on the given board is solved or not
 - `find_empty_cell` helper function that helps to find an empty cell on a given board

In [ ]:
class Stack:
    def __init__(self):
        self.stack = []

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        if not self.is_empty():
            return self.stack.pop()
        else:
            return None

    def is_empty(self):
        return len(self.stack) == 0


def dfs_solve(board):
    stack = Stack()
    stack.push(board)

    while not stack.is_empty():
        current_board = stack.pop()

        # Check if the board is solved
        if is_board_solved(current_board):
            print("Solution found:")
            print_board(current_board)
            return current_board

        # Find an empty cell on the board
        empty_cell = find_empty_cell(current_board)
        if empty_cell is None:
            continue  # No empty cells found, backtrack

        row, col = empty_cell

        # Try all possible numbers in the empty cell
        for num in range(1, 5):
            if valid_move(current_board, (row, col, num)):
                # Make a copy of the current board and update it with the new move
                new_board = [row[:] for row in current_board]
                update_board(new_board, row, col, str(num))
                stack.push(new_board)

    print("No solution found.")
    return None


def is_board_solved(board):
    for row in board:
        if " " in row:
            return False
    return True


def find_empty_cell(board):
    for i in range(4):
        for j in range(4):
            if board[i][j] == " ":
                return i, j
    return None

### 4.Depth-limited search
 - In certain scenarios, the depth limit chosen for DLS may be too small to reach a solution, resulting in a failure to find a solution even though one exists at a deeper level. Conversely, if the depth limit is set too high, DLS effectively behaves the same as DFS, exploring the entire state space until a solution is found or until the maximum depth is reached.

 - While DLS can be useful in scenarios where the depth of the state space is known and the search needs to be bounded within a certain depth level, it may not be suitable for solving problems with unknown or variable-depth state spaces like Sudoku puzzles.

### 5. Iterative deepening search
- Iterative Deepening Search is the same as a Depth-Limited Search, but with a dynamically changing depth limit. This addresses the variable-state depth limitation that Depth-Limited Search has.
- A problem arises when applying this in the case of sudoku, as the goal node is guaranteed to only be found at the lowest depth, signifying a board with every position filled. This means that the iterative deepening will always deepen all the way to the lowest depth, and as discussed above, makes the algorithm effectively the same as a DFS.


### 6. Greedy Search

- An algorithm goes through every position and counts the number of possible answers in each position of the board.

- The number of possible answers in each position serves as a heuristic for the Depth First Search Algorithm, and the search algorithm works on the least positions with the least number of possible answers first.

- This reduces the amount of times the DFS has to backtrack when finding for the solution.

In [ ]:
from collections import OrderedDict

def valid_move(board, move):
  valid = True
  row, col, num = move
  num = str(num)
  for i in range(4):
    if (board[row][i] == num):
      valid = False
      return valid
  for j in range(4):
    if (board[j][col] == num):
      valid = False
      return valid

  # Check if the number already exists in the 2x2 square
  start_row = (row // 2) * 2  # Find the starting row index of the square
  start_col = (col // 2) * 2  # Find the starting column index of the square
  for i in range(start_row, start_row + 2):
      for j in range(start_col, start_col + 2):
          if board[i][j] == num:
            valid = False
            return valid

  return valid

def allowedNumbers(board):

  allow_list = {}

  for i in range(4):
    for j in range(4):
      for k in range(4):
        move = (i,j,k)
        if (valid_move(board,move) == True):
          allow_list[(i,j)]=[]
          allow_list[(i,j)].append(k)

  return allow_list;

def greedy_solve(board):

    stack = Stack()
    heur = allowedNumbers(board)
    #sort dict by list length
    res = OrderedDict(sorted(heur.items(), key = lambda x : len(x[1]))).keys()

    for item in res :
      stack.push(board)

    while not stack.is_empty():
        current_board = stack.pop()

        # Check if the board is solved
        if is_board_solved(current_board):
            print("Solution found:")
            print_board(current_board)
            return current_board

        # Find an empty cell on the board
        empty_cell = find_empty_cell(current_board)
        if empty_cell is None:
            continue  # No empty cells found, backtrack

        row, col = empty_cell

        # Try all possible numbers in the empty cell
        for num in range(1, 5):
            if valid_move(current_board, (row, col, num)):
                # Make a copy of the current board and update it with the new move
                new_board = [row[:] for row in current_board]
                update_board(new_board, row, col, str(num))
                stack.push(new_board)

    print("No solution found.")
    return None


### 7. A* Search

 - A* Search is a mixture of Greedy Search and Uniform Cost Search. However, as earlier discussed, in sudoku, the path cost between nodes is uniform since each step or transition between states (placing a number in a cell) has the same cost. This reduces the A* search into simply a Greedy Search, which is demonstrated above.

### **The agent:**

In [ ]:
def sudoku_agent():
  ''' The main function for the agent
  '''
  # Introduction
  print("Welcome to 4X4 sudoku solver, please input a valid sudoku puzzle to the agent")
  print("To end your input, type in -1")

  # Initiate the game
  game_board = create_board()
  inputBoard(game_board)
  print_board(game_board)

  # Solving the game
  solution = 0
  solution = int(input("Select the search algorithm you want to use: 1-DFS, 2-Greedy Search"))
  if solution == 1:
    dfs_solve(game_board)
  elif solution == 2:
    greedy_solve(game_board)
  else:
    print("Invalid input.")

### **Testing:**

In [ ]:
sudoku_agent()

Welcome to 4X4 sudoku solver, please input a valid sudoku puzzle to the agent
To end your input, type in -1
Input the row (1-4): 1
Input the column (1-4): 1
Input the number (1-4): 1
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 2
Input the column (1-4): 2
Input the number (1-4): 3
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 2
Input the column (1-4): 3
Input the number (1-4): 1
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 2
Input the column (1-4): 4
Input the number (1-4): 2
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 3
Input the column (1-4): 1
Input the number (1-4): 2
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 3
Input the column (1-4): 3
Input the number (1-4): 4
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 4
Input the column (1-4): 1
Input the number (1-4): 3
To continue input, choose

In [ ]:
sudoku_agent()

Welcome to 4X4 sudoku solver, please input a valid sudoku puzzle to the agent
To end your input, type in -1
Input the row (1-4): 1
Input the column (1-4): 1
Input the number (1-4): 1
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 2
Input the column (1-4): 2
Input the number (1-4): 3
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 2
Input the column (1-4): 3
Input the number (1-4): 1
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 3
Input the column (1-4): 1
Input the number (1-4): 2
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 2
Input the column (1-4): 4
Input the number (1-4): 2
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 3
Input the column (1-4): 3
Input the number (1-4): 4
To continue input, choose 0. To stop input, choose -1: 0
Input the row (1-4): 4
Input the column (1-4): 1
Input the number (1-4): 3
To continue input, choose